In [1]:
from symred.chem import xyz_from_pubchem
import numpy as np
from openfermion import FermionOperator
from scipy.sparse.linalg import expm
from openfermion import get_sparse_operator

In [2]:
mol_name = 'H2O'
# # mol_name = 'H2'

# mol_name = 'LiH'

In [3]:
xyz_file = xyz_from_pubchem(mol_name)
print(xyz_file)

3
 
O	0	0	0
H	0.2774	0.8929	0.2544
H	0.6068	-0.2383	-0.7169



In [4]:
from symred.chem import PySCFDriver

In [5]:
basis = 'STO-3G'
convergence = 1e6
charge=0
max_hf_cycles=50
ram = 8_000
run_mp2  = True,
run_cisd = True,
run_ccsd = True,
run_fci  = True

In [6]:
pyscf_obj = PySCFDriver(xyz_file,
                       basis,
                       convergence=convergence,
                       charge=charge,
                       max_ram_memory=ram,
                       max_hf_cycles=max_hf_cycles,
                       
                       run_mp2=run_mp2,
                       run_cisd=run_cisd,
                       run_ccsd=run_ccsd,
                       run_fci=run_fci)

In [7]:
pyscf_obj.run_pyscf()

In [8]:
2*pyscf_obj.pyscf_hf.mol.nao

14

In [9]:
from symred.chem import FermionicHamilt

In [10]:
H_ferm = FermionicHamilt(pyscf_obj.pyscf_hf)

H_ferm.build_operator()
# H_ferm.fermionic_molecular_hamiltonian

In [11]:
hf_state = H_ferm.hf_comp_basis_state
hf_state


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0])

In [17]:
hf_ket = H_ferm.hf_ket
hf_ket.shape

(16384,)

In [12]:
print(H_ferm.scf_method.energy_tot())
print(pyscf_obj.pyscf_hf.energy_tot())

H_mat = H_ferm.get_sparse_ham()
hf_ket.conj().T @ H_mat @ hf_ket

-74.96386325726527
-74.96386325726527


NameError: name 'hf_ket' is not defined

In [ ]:
one_body_integrals = H_ferm._one_body_integrals
two_body_integrals = H_ferm._two_body_integrals

g = two_body_integrals
fij = pyscf_obj.pyscf_hf.mo_energy

nocc = H_ferm.n_electrons // 2  # this is never the active space
e_orbs_occ = fij[:nocc]
e_i = e_orbs_occ.reshape(1, 1, -1, 1)
e_j = e_orbs_occ.reshape(1, 1, 1, -1)

e_orbs_vir = fij[nocc:]
e_a = e_orbs_vir.reshape(-1, 1, 1, 1)
e_b = e_orbs_vir.reshape(1, -1, 1, 1)

abgij = g[nocc:, nocc:, :nocc, :nocc]
amplitudes = abgij * 1.0 / (e_i + e_j - e_a - e_b)
E = 2.0 * np.einsum('abij,abij->', amplitudes, abgij) - np.einsum('abji,abij', amplitudes, abgij,
                                                                        optimize='greedy')

E

In [53]:
F_MO@ state_new

array([-2.02456292e+01, -1.29168653e+00, -6.16741573e-01, -4.92519800e-01,
       -3.99932004e-01, -1.31278880e-02, -4.49575683e-03])

In [44]:
pyscf_obj.pyscf_hf.mo_energy

array([-19.99076276,  -1.17811723,  -0.56047599,  -0.35032754,
        -0.2639027 ,   0.63476246,   0.76806442])

In [39]:
sum(np.diag(F_MO[:5]))

-23.008080140342994

In [47]:
pyscf_obj.pyscf_hf.mol.nelectron//2

5

In [56]:
F_MO.shape[0]

7

In [71]:
F_MO = pyscf_obj.pyscf_hf.mo_coeff.T @  pyscf_obj.pyscf_hf.get_fock() @ pyscf_obj.pyscf_hf.mo_coeff

two_body_integrals = np.zeros((F_MO.shape[0],F_MO.shape[0],F_MO.shape[0],F_MO.shape[0]))

from openfermion import InteractionOperator #, get_sparse_operator, FermionOperator
from openfermion.chem.molecular_data import spinorb_from_spatial

one_body_coefficients, two_body_coefficients = spinorb_from_spatial(
    F_MO, two_body_integrals
)

FF = InteractionOperator(0,
                      one_body_coefficients,
                      two_body_coefficients)



In [185]:
J = pyscf_obj.pyscf_hf.get_j()
K = pyscf_obj.pyscf_hf.get_k()
hcore = pyscf_obj.pyscf_hf.get_hcore()
pyscf_obj.pyscf_hf.get_fock() ==  hcore + (J-0.5*K) 

array([[ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True]])

In [186]:
JK_MO = pyscf_obj.pyscf_hf.mo_coeff.T @  (J/2-K/4) @ pyscf_obj.pyscf_hf.mo_coeff
F_MO = pyscf_obj.pyscf_hf.mo_coeff.T @  pyscf_obj.pyscf_hf.get_fock() @ pyscf_obj.pyscf_hf.mo_coeff
F_MO = F_MO - JK_MO

two_body_integrals = np.zeros((F_MO.shape[0],F_MO.shape[0],F_MO.shape[0],F_MO.shape[0]))

from openfermion import InteractionOperator #, get_sparse_operator, FermionOperator
from openfermion.chem.molecular_data import spinorb_from_spatial

one_body_coefficients, two_body_coefficients = spinorb_from_spatial(
    F_MO, two_body_integrals
)

FF = InteractionOperator(0,
                      one_body_coefficients,
                      two_body_coefficients)

H_ferm.hf_ket.T @ get_sparse_operator(FF) @ H_ferm.hf_ket

(-84.04822776924128+0j)

In [198]:
V = get_sparse_operator(H_ferm.fermionic_molecular_hamiltonian) - get_sparse_operator(FF)

In [199]:
H_ferm.hf_ket.T @ get_sparse_operator(FF) @ H_ferm.hf_ket + H_ferm.hf_ket.T @ V @ H_ferm.hf_ket

(-74.96386325726526+0j)

In [200]:
pyscf_obj.pyscf_mp2.e_tot

-75.00118490739871

In [183]:
F_MO = pyscf_obj.pyscf_hf.mo_coeff.T @  (hcore + J/3 -K ) @ pyscf_obj.pyscf_hf.mo_coeff
# F_MO = pyscf_obj.pyscf_hf.mo_coeff.T @  pyscf_obj.pyscf_hf.get_fock() @ pyscf_obj.pyscf_hf.mo_coeff
# F_MO = F_MO - JK_MO

two_body_integrals = np.zeros((F_MO.shape[0],F_MO.shape[0],F_MO.shape[0],F_MO.shape[0]))

from openfermion import InteractionOperator #, get_sparse_operator, FermionOperator
from openfermion.chem.molecular_data import spinorb_from_spatial

one_body_coefficients, two_body_coefficients = spinorb_from_spatial(
    F_MO, two_body_integrals
)

FF = InteractionOperator(0,
                      one_body_coefficients,
                      two_body_coefficients)

H_ferm.hf_ket.T @ get_sparse_operator(FF) @ H_ferm.hf_ket

(-127.00077278102881+0j)

In [143]:
F_MO = pyscf_obj.pyscf_hf.mo_coeff.T @  pyscf_obj.pyscf_hf.get_fock() @ pyscf_obj.pyscf_hf.mo_coeff
J_MO = pyscf_obj.pyscf_hf.mo_coeff.T @  pyscf_obj.pyscf_hf.get_k() @ pyscf_obj.pyscf_hf.mo_coeff

F_MO = F_MO  - 2*J_MO

two_body_integrals = np.zeros((F_MO.shape[0],F_MO.shape[0],F_MO.shape[0],F_MO.shape[0]))

from openfermion import InteractionOperator #, get_sparse_operator, FermionOperator
from openfermion.chem.molecular_data import spinorb_from_spatial

one_body_coefficients, two_body_coefficients = spinorb_from_spatial(
    F_MO, two_body_integrals
)

FF = InteractionOperator(0,
                      one_body_coefficients,
                      two_body_coefficients)



In [146]:
H_ferm.hf_ket.T @ get_sparse_operator(FF) @ H_ferm.hf_ket

(18.165113509361472+0j)

In [148]:
print(pyscf_obj.pyscf_hf.energy_elec())
sum(pyscf_obj.pyscf_hf.energy_elec())

(-84.04822776924131, 38.0320674885553)


-46.01616028068601

In [111]:
sum(pyscf_obj.pyscf_hf.energy_elec()) - H_ferm.hf_ket.T @ get_sparse_operator(FF) @ H_ferm.hf_ket

(23.557312121617983+0j)

In [92]:
H_ferm.hf_ket.T @ get_sparse_operator(FF) @ H_ferm.hf_ket

(-46.01616028068598+0j)

In [36]:
ind = 5
pyscf_obj.pyscf_hf.mo_coeff[:,ind].T @ F_MO @ pyscf_obj.pyscf_hf.mo_coeff[:,ind]

-0.8146034351637497

In [23]:
state_new = pyscf_obj.pyscf_hf.mo_occ//2

In [19]:
np.ones

(7, 7)

In [ ]:
ijgij = g[:nocc, :nocc, :nocc, :nocc]
h_ij = one_body_integrals[:nocc, :nocc]
g_ijji = np.einsum('ijkl->ijlk',ijgij)

np.einsum('ii->', h_ij) + (np.einsum('ijij->',ijgij)  - 2*np.einsum('ijji->', ijgij))

In [136]:
fij

array([-19.99076276,  -1.17811723,  -0.56047599,  -0.35032754,
        -0.2639027 ,   0.63476246,   0.76806442])

In [ ]:
one_body_integrals = H_ferm._one_body_integrals
two_body_integrals = H_ferm._two_body_integrals

g = two_body_integrals
fij = pyscf_obj.pyscf_hf.mo_energy

nocc = H_ferm.n_electrons // 2  # this is never the active space
e_orbs_occ = fij[:nocc]
e_i = e_orbs_occ.reshape(1, 1, -1, 1)
e_j = e_orbs_occ.reshape(1, 1, 1, -1)

e_orbs_vir = fij[nocc:]
e_a = e_orbs_vir.reshape(-1, 1, 1, 1)
e_b = e_orbs_vir.reshape(1, -1, 1, 1)

abgij = g[nocc:, nocc:, :nocc, :nocc]